In [1]:
#!pip install onnx_tf
!pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import json
import os
import gc
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torchinfo import summary
import onnx
import tflite_runtime.interpreter as tflite

In [3]:
#train = pd.read_csv('/kaggle/input/asl-signs/train.csv')

In [4]:
import torch
from torch import nn
import torch.nn.functional as F

from typing import Dict, Optional
 
import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor



# Torch model and weights

In [5]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=4c6f15c5b739a1de58f6ea67280c7ccb4e4165547ac2e7acd822cba42bc096b6
  Stored in directory: /root/.cache/pip/wheels/96/3f/5f/13976445f67f3b4e77b054e65f7f4c39016e92e8358fe088db
Successfully built efficientnet_pytorch


In [6]:
from distutils.dir_util import copy_tree
copy_tree("/kaggle/input/keras-repo-v2/utils", "utils")

['utils/.idea/.gitignore',
 'utils/.idea/utils.iml',
 'utils/.idea/modules.xml',
 'utils/.idea/inspectionProfiles/profiles_settings.xml',
 'utils/.idea/inspectionProfiles/Project_Default.xml',
 'utils/.idea/misc.xml',
 'utils/keras_models/preprocess.py',
 'utils/keras_models/efficientnet.py',
 'utils/keras_models/transfer.py',
 'utils/keras_models/common.py',
 'utils/keras_models/rexnet.py',
 'utils/keras_models/transformer.py',
 'utils/keras_models/__init__.py',
 'utils/augmentations.py',
 'utils/__init__.py']

# Set params

In [7]:
def transfer_efficientnet_artem(torch_model_path, model_name, size = (160, 80)):
    stt = torch.load(torch_model_path, map_location = "cpu")["model"]
    stt = dict([(n.split('base_model.')[-1], p) for n, p in stt.items() if 'hyper' not in n])
    model = EfficientNet.from_name('efficientnet-b0', num_classes=250, in_channels=3)
    model.eval()
    model.load_state_dict(stt)


    input_layer = K.layers.Input((*size, 3), batch_size = 1, name = "inputs")
    model_layer = KerasEfficientNet(model, name = "model")(input_layer)
    output_layer = KerasIdentity(None, name = "outputs")(model_layer)
    keras_model = K.models.Model(input_layer, output_layer, name = model_name)

    newstt = {k[1:].replace("._", "."): v for k, v in model.state_dict().items()}

    original_names = [_.name for _ in keras_model.weights]
    names = [replace(_.name)[:-2] for _ in keras_model.weights]
    shapes = {replace(_.name)[:-2]: _.numpy().shape for _ in keras_model.weights}
    for name, original_name in zip(names, original_names):
        if modify(name, original_name, newstt).shape != shapes[name]:
            print(name, ", ", modify(name, original_name, newstt).shape, "-->", shapes[name])

    keras_model.set_weights([modify(name, original_name, newstt) for name, original_name in zip(names, original_names)])
    return model, keras_model

In [8]:
from utils.keras_models.preprocess import KerasPreprocessing, KerasTransformerPreprocess
from utils.keras_models.transfer import *

#####################################################################################
INPUT_SIZE = (160, 80) #(160, 80)

# WEIGHTs FOR CNN - TRANSFORMER
CNN_TRANS_WGT = (0.6, 0.4)

# CHOOSE CNN: 'rex' or 'eff'
CNN = 'eff'
# SET NUMBER OF TRANSFORMER MODELS: 3 or 4
NUM_TRANS = 2
print(f'Amount of transformers: {NUM_TRANS}')

#####################################################################################

################# CNN
#effckpt = '/kaggle/input/islr-share/8963/partialaug_OPT_effb0x160x80_8f_f0_0_bs64_sm0.58_norm1_inter0.23_time13_freq80_repl0.11_lr0.002391_mixup0.42-4.16_aug0.18_rot0.25_zero0.00_shift0.13_scale0.25_inver0.28_ep179/checkpoints/img_v0_fold0_final_loop_best_main_0.8963_0.9503.pth'

# FULL TRAIN CKPT
#effckpt = '/kaggle/input/full-data-train/_FULL_TRAIN_1_8963params/checkpoints/img_v0_FULL_ep179.pth'

# NEW effb0 f8
effckpt = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.82_pwcom1.40_sm0.51_inter0.37_repl0.24_lr0.002705_mixup0.49_aug0.24_rot0.17_shift0.20_scale0.31_inver0.22_ep182/checkpoints/img_v0_fold0_final_loop_best_main_0.8986_0.9518.pth'
#effckpt = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.81_pwcom0.86_sm0.54_inter0.32_repl0.16_lr0.002544_mixup0.46_aug0.26_rot0.11_shift0.21_scale0.28_inver0.32_ep179/checkpoints/img_v0_fold0_final_loop_best_main_0.8975_0.9531.pth'
#effckpt = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_1_pwbad1.04_pwcom1.63_sm0.51_inter0.36_repl0.13_lr0.002719_mixup0.44_aug0.21_rot0.34_shift0.31_scale0.25_inver0.24_ep178/checkpoints/img_v0_fold0_final_loop_best_main_0.8966_0.9517.pth'


artem_ckpt = '/kaggle/input/160x80-weights/twohand_OPT_HYP_SCEX_0.5_DROPLAST/checkpoints/img_v0_fold0_final_loop_best_main_0.8992_0.9513.pth'
#eff b0

torch_cnn, keras_cnn = transfer_efficientnet(effckpt, "eff", size=INPUT_SIZE)
input_layer = K.layers.Input((543, 3), name = "inputs")
preprocess_layer = KerasPreprocessing(size=INPUT_SIZE, name = "preprocess")(input_layer)
cnn_out = keras_cnn(preprocess_layer)
keras_cnn_model = K.models.Model(input_layer, cnn_out)
tf.saved_model.save(keras_cnn_model, 'effb0_160')

'''torch_cnn, keras_cnn = transfer_efficientnet_artem(artem_ckpt, "eff", size=INPUT_SIZE)
input_layer = K.layers.Input((543, 3), name = "inputs")
preprocess_layer = KerasPreprocessing(size=INPUT_SIZE, name = "preprocess")(input_layer)
cnn_out = keras_cnn(preprocess_layer)
keras_cnn_model = K.models.Model(input_layer, cnn_out)
tf.saved_model.save(keras_cnn_model, 'effb0_160_artem')'''


'''#16f ckpts
effckpt16 = '/kaggle/input/effb0-f16-x160x80/n1_effb0_x160x80_16f_f0_2_pwbad0.92_pwcom0.81_sm0.54_inter0.22_repl0.11_lr0.002575_mixup0.65_aug0.16_rot0.24_shift0.16_scale0.23_inver0.30_ep172/checkpoints/img_v0_fold0_final_loop_best_main_0.8986_0.9522.pth'
#effckpt16 = '/kaggle/input/effb0-f16-x160x80/n1_effb0_x160x80_16f_f0_2_pwbad1.24_pwcom0.92_sm0.54_inter0.23_repl0.09_lr0.002272_mixup0.68_aug0.17_rot0.26_shift0.17_scale0.23_inver0.27_ep185/checkpoints/img_v0_fold0_final_loop_best_main_0.8992_0.9522.pth'

torch_cnn, keras_cnn = transfer_efficientnet(effckpt16, "eff", size=INPUT_SIZE)
input_layer = K.layers.Input((543, 3), name = "inputs")
preprocess_layer = KerasPreprocessing(size=INPUT_SIZE, name = "preprocess")(input_layer)
cnn_out = keras_cnn(preprocess_layer)
keras_cnn_model = K.models.Model(input_layer, cnn_out)
tf.saved_model.save(keras_cnn_model, 'effb0_160x16f')'''



'''rexckpt = "/kaggle/input/rexnet-ckpts/timm_fold0_final_loop_best_main_0.8953_0.9499.pth" # 160
#rexckpt = "/kaggle/input/rexnet-ckpts/timm_fold0_ep165_acc0.8950.pth" # 128
#rexnet
torch_cnn, keras_cnn = transfer_rexnet(rexckpt, "rex", size=INPUT_SIZE)

input_layer = K.layers.Input((543, 3), name = "inputs")
preprocess_layer = KerasPreprocessing(size=(160, 80), name = "preprocess")(input_layer)
cnn_out = keras_cnn(preprocess_layer)
keras_cnn_model = K.models.Model(input_layer, cnn_out)
tf.saved_model.save(keras_cnn_model, 'rex160')'''


################# BERT
bert_full, keras_full = transfer_transformer("/kaggle/input/islr-share/transformers_weights/bert_full_data.ckpt", "bert_full")
bert_configf0, keras_bertf0 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/bert_f0.ckpt", "bertf0")
bert_configf1, keras_bertf1 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/bert_f1.ckpt", "bertf1")

bert_configf0x16, keras_bertf0x16 = transfer_transformer("/kaggle/input/islr-share/bert_16f0.ckpt", "bertf0x16")


tf.saved_model.save(keras_full, 'bert_full')
tf.saved_model.save(keras_bertf0, 'bert_f0')
tf.saved_model.save(keras_bertf1, 'bert_f1')
tf.saved_model.save(keras_bertf0x16, 'bert_f0x16')


################# DEBERTA
deberta_config, keras_deberta = transfer_transformer("/kaggle/input/islr-share/transformers_weights/deberta_full_data.ckpt", "deberta")
deberta_configf0, keras_debertaf0 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/deberta_f0.ckpt", "debertaf0")
deberta_configf1, keras_debertaf1 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/deberta_f1.ckpt", "debertaf1")
deberta_configf0x16, keras_debertaf0x16 = transfer_transformer("/kaggle/input/islr-share/deberta_16f0.ckpt", "debertaf0x16")


tf.saved_model.save(keras_deberta,'deberta_full')
tf.saved_model.save(keras_debertaf0, 'deberta_f0')
tf.saved_model.save(keras_debertaf1, 'deberta_f1')
tf.saved_model.save(keras_debertaf0x16, 'deberta_f0x16')


################# TRANS PREPROC
input_layer = K.layers.Input((543, 3), name = "inputs")
transformer_preprocess_layer = KerasTransformerPreprocess(bert_full, name="trans_preprocess")(input_layer)
keras_tr_prep = K.models.Model(input_layer, transformer_preprocess_layer)
tf.saved_model.save(keras_tr_prep, 'keras_tr_prep')
print('done')

Amount of transformers: 2


/kaggle/working/utils/keras_models/transfer.py:34: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  w = w.permute(2, 3, 1, 0) if len(w.shape) == 4 else w.T


done


In [9]:
import time
base_dir = '/kaggle/input/asl-signs/'
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

train = pd.read_csv(base_dir + '/train.csv')
yy = load_relevant_data_subset(base_dir + train.loc[45, 'path'])

# TFLite Model

In [10]:
from tensorflow import keras

class TFLiteModel(tf.Module):

    def __init__(self):
        super(TFLiteModel, self).__init__()
        
        self.cnn = tf.saved_model.load('effb0_160')
        ####self.cnn = tf.saved_model.load('effb0_160_artem')
        ####self.cnn = tf.saved_model.load('rex160')        
        #self.cnn = tf.saved_model.load('effb0_160x16f')
        
        
        self.tr_prep = tf.saved_model.load('keras_tr_prep')
        
        self.bertfull = tf.saved_model.load('bert_full')
        #self.bertf0 = tf.saved_model.load('bert_f0')
        #self.bertf1 = tf.saved_model.load('bert_f1')        
        #self.bertf0x16 = tf.saved_model.load('bert_f0x16')
        
        self.debertafull = tf.saved_model.load('deberta_full')
        #self.debertaf0 = tf.saved_model.load('deberta_f0')
        #self.debertaf1 = tf.saved_model.load('deberta_f1')        
        #self.debertaf0x16 = tf.saved_model.load('deberta_f0x16')

        # artem hyper 0.9045723962743438 (0.58, 0.42000000000000004, 0.65, 0.35)
        # 0.90042 (0.57, 0.43, 0.7, 0.3) our current best effb0 with 2 trans
        
        # (f16 8992) 0.9036409822184589 (0.65, 0.35, 0.8, 0.19999999999999996)
        # (f16 8986) 0.9048264182895851 (0.63, 0.37, 0.6000000000000001, 0.3999999999999999)
        
        # 8986  0.904233700254022 (0.58, 0.42000000000000004, 0.8500000000000001, 0.1499999999999999)
        # 8975  0.9022015241320914 (0.56, 0.43999999999999995, 0.8500000000000001, 0.1499999999999999)
        # 8966  0.9022015241320914 (0.5700000000000001, 0.42999999999999994, 0.45, 0.55)
        
        self.wgt = (0.58, 0.42) 
        
        self.bert_wgt = 0.85   
        self.deberta_wgt = 0.15

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        
        cnn_out = self.cnn(inputs)
        
        tr_input = self.tr_prep(inputs)
        
        bertfull = self.bertfull(tr_input)
        #bertf0 = self.bertf0(tr_input)
        #bertf1 = self.bertf1(tr_input)
        #bertf0x16 = self.bertf0x16(tr_input)
        
        debertafull = self.debertafull(tr_input)
        #debertaf0 = self.debertaf0(tr_input)
        #debertaf1 = self.debertaf1(tr_input)
        #debertaf0x16 = self.debertaf0x16(tr_input)
        
        trans_out = tf.add_n([
                            tf.multiply(bertfull, self.bert_wgt),
                            #tf.multiply(bertf0, self.bert_wgt),
                            #tf.multiply(bertf1, self.bert_wgt),
                            #tf.multiply(bertf0x16, self.bert_wgt),
            
                            tf.multiply(debertafull, self.deberta_wgt),
                            #tf.multiply(debertaf0, self.deberta_wgt),
                            #tf.multiply(debertaf1, self.deberta_wgt),
                            #tf.multiply(debertaf0x16, self.deberta_wgt),
                            ])        
        
        out = tf.add_n([tf.multiply(cnn_out, self.wgt[0]), tf.multiply(trans_out, self.wgt[1])])
        return {'outputs': out}      

tflite_keras_model = TFLiteModel()
st = time.time()
demo_output = tflite_keras_model(yy)["outputs"]
print(np.argmax(demo_output))
print(time.time() - st)
#decoder(np.argmax(demo_output.numpy(), axis=-1))

48
4.784736394882202


In [11]:
st = time.time()
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)

use_FP16 = False
if use_FP16: #(CNN == 'eff' and NUM_TRANS > 2) or NUM_TRANS > 3:
    print(f'Using FP16')
    keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]
    keras_model_converter.target_spec.supported_types = [tf.float16]

tflite_model = keras_model_converter.convert()
with open('/kaggle/working/model_FINAL.tflite', 'wb') as f:
    f.write(tflite_model)
!zip submission.zip /kaggle/working/model_FINAL.tflite

print(f'Done with {(time.time() - st)/60:.2f} minutes')

  adding: kaggle/working/model_FINAL.tflite (deflated 8%)
Done with 0.97 minutes


In [12]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter('/kaggle/working/model_FINAL.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

st = time.time()
output = prediction_fn(inputs=yy)
print(time.time() - st)
sign = np.argmax(output["outputs"])

print(sign)

0.14821624755859375
48


In [13]:
import json
def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

label_index = read_dict(f"/kaggle/input/asl-signs/sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
train["label"] = train["sign"].map(lambda sign: label_index[sign])
print(train.shape)
display(train.head())

(94477, 5)


,path,participant_id,sequence_id,sign,label
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,25
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,232
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,48
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,23
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,164


# Timings

In [14]:
st = time.time()
cnt = 0
total = 115
model_time = 0

for i in tqdm(np.linspace(0, len(train)-1, total, dtype=int)):
    sample = train.loc[i] # 45 len=225

    yy = load_relevant_data_subset(base_dir + sample['path'])
    md_st = time.time()
    output = prediction_fn(inputs=yy)
    model_time += time.time() - md_st
    sign = np.argmax(output["outputs"])
    #print(sample['sign'], sample['label'], sign)
    
    if sample['label'] == sign: cnt +=1

print(f'accuracy: {cnt / total:.5f}')  

print(f'Mean time: {(time.time() - st)/total:.7f}')
print(f'Mean time only infer: {model_time/total:.7f}')

100%|██████████| 115/115 [00:10<00:00, 11.23it/s]

accuracy: 1.00000
Mean time: 0.0891357
Mean time only infer: 0.0660898


In [15]:
'''import tflite_runtime.interpreter as tflite
import time

from utils.keras_models.preprocess import KerasPreprocessing, KerasTransformerPreprocess
from utils.keras_models.transfer import *

INPUT_SIZE = (160, 80)

#here_cnn = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.82_pwcom1.40_sm0.51_inter0.37_repl0.24_lr0.002705_mixup0.49_aug0.24_rot0.17_shift0.20_scale0.31_inver0.22_ep182/checkpoints/img_v0_fold0_final_loop_best_main_0.8986_0.9518.pth'
#here_cnn = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.81_pwcom0.86_sm0.54_inter0.32_repl0.16_lr0.002544_mixup0.46_aug0.26_rot0.11_shift0.21_scale0.28_inver0.32_ep179/checkpoints/img_v0_fold0_final_loop_best_main_0.8975_0.9531.pth'
here_cnn = '/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_1_pwbad1.04_pwcom1.63_sm0.51_inter0.36_repl0.13_lr0.002719_mixup0.44_aug0.21_rot0.34_shift0.31_scale0.25_inver0.24_ep178/checkpoints/img_v0_fold0_final_loop_best_main_0.8966_0.9517.pth'
torch_cnn, keras_cnn = transfer_efficientnet(here_cnn, "eff", size=INPUT_SIZE)
input_layer = K.layers.Input((543, 3), name = "inputs")
preprocess_layer = KerasPreprocessing(size=INPUT_SIZE, name = "preprocess")(input_layer)
cnn_out = keras_cnn(preprocess_layer)
keras_cnn_model = K.models.Model(input_layer, cnn_out)
tf.saved_model.save(keras_cnn_model, 'effb0_160_test')

cnn_md = tf.saved_model.load('effb0_160_test')

cnn_conver = tf.lite.TFLiteConverter.from_keras_model(cnn_md)

tflite_model = cnn_conver.convert()
with open('/kaggle/working/cnn_conver.tflite', 'wb') as f:
    f.write(tflite_model)
    
interpreter = tflite.Interpreter('/kaggle/working/cnn_conver.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn_cnn = interpreter.get_signature_runner("serving_default")


bert_configf0, keras_bertf0 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/bert_f0.ckpt", "bertf0")
deberta_configf0, keras_debertaf0 = transfer_transformer("/kaggle/input/islr-share/transformers_weights/deberta_f0.ckpt", "debertaf0")

#bert_configf0, keras_bertf0 = transfer_transformer("/kaggle/input/islr-share/bert_16f0.ckpt", "bertf0")
#deberta_configf0, keras_debertaf0 = transfer_transformer("/kaggle/input/islr-share/deberta_16f0.ckpt", "debertaf0")


input_layer = K.layers.Input((543, 3), name = "inputs")
transformer_preprocess_layer = KerasTransformerPreprocess(bert_configf0, name="trans_preprocess")(input_layer)
bertf0_out = keras_bertf0(transformer_preprocess_layer)
debertaf0_out = keras_debertaf0(transformer_preprocess_layer)

output_layer = KerasIdentity(None, name = "outputs")((bertf0_out + debertaf0_out) / 2)
carno = K.models.Model(input_layer, output_layer)

trans_conver = tf.lite.TFLiteConverter.from_keras_model(carno)

tflite_model = trans_conver.convert()
with open('/kaggle/working/trans_conver.tflite', 'wb') as f:
    f.write(tflite_model)
    
interpreter = tflite.Interpreter('/kaggle/working/trans_conver.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn_trans = interpreter.get_signature_runner("serving_default")

########################################################
input_layer = K.layers.Input((543, 3), name = "inputs")
transformer_preprocess_layer = KerasTransformerPreprocess(bert_configf0, name="trans_preprocess")(input_layer)
bertf0_out = keras_bertf0(transformer_preprocess_layer)
output_layer = KerasIdentity(None, name = "outputs")(bertf0_out)
bertf0__ = K.models.Model(input_layer, output_layer)
trans_conver11 = tf.lite.TFLiteConverter.from_keras_model(bertf0__)

tflite_model = trans_conver11.convert()
with open('/kaggle/working/bertf0_test.tflite', 'wb') as f:
    f.write(tflite_model)
    
interpreter = tflite.Interpreter('/kaggle/working/bertf0_test.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn_trans11 = interpreter.get_signature_runner("serving_default")

input_layer = K.layers.Input((543, 3), name = "inputs")
transformer_preprocess_layer = KerasTransformerPreprocess(bert_configf0, name="trans_preprocess")(input_layer)
debertaf0_out = keras_debertaf0(transformer_preprocess_layer)
output_layer = KerasIdentity(None, name = "outputs")(debertaf0_out)
debertf0__ = K.models.Model(input_layer, output_layer)
trans_conver22 = tf.lite.TFLiteConverter.from_keras_model(debertf0__)

tflite_model = trans_conver22.convert()
with open('/kaggle/working/debertf0_test.tflite', 'wb') as f:
    f.write(tflite_model)
    
interpreter = tflite.Interpreter('/kaggle/working/debertf0_test.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn_trans22 = interpreter.get_signature_runner("serving_default")


print('done')'''

'import tflite_runtime.interpreter as tflite\nimport time\n\nfrom utils.keras_models.preprocess import KerasPreprocessing, KerasTransformerPreprocess\nfrom utils.keras_models.transfer import *\n\nINPUT_SIZE = (160, 80)\n\n#here_cnn = \'/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.82_pwcom1.40_sm0.51_inter0.37_repl0.24_lr0.002705_mixup0.49_aug0.24_rot0.17_shift0.20_scale0.31_inver0.22_ep182/checkpoints/img_v0_fold0_final_loop_best_main_0.8986_0.9518.pth\'\n#here_cnn = \'/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_0_pwbad0.81_pwcom0.86_sm0.54_inter0.32_repl0.16_lr0.002544_mixup0.46_aug0.26_rot0.11_shift0.21_scale0.28_inver0.32_ep179/checkpoints/img_v0_fold0_final_loop_best_main_0.8975_0.9531.pth\'\nhere_cnn = \'/kaggle/input/160x80-weights/eff_HYP_SCEX_DROPLAST_0.5_5_Rx160x80_8f_f0_1_pwbad1.04_pwcom1.63_sm0.51_inter0.36_repl0.13_lr0.002719_mixup0.44_aug0.21_rot0.34_shift0.31_scale0.25_inver0.24_ep178/checkpoints/img_v0_fold0

In [16]:
'''
import json
def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

class CFG:
    base_path = '/kaggle/input/'
    fold_group = False
    n_fold = 8 #8
train = pd.read_csv('/kaggle/input/asl-signs/train.csv')
label_index = read_dict(f"{CFG.base_path}/asl-signs/sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
train["label"] = train["sign"].map(lambda sign: label_index[sign])
print(train.shape)
display(train.head())

import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold

split = StratifiedKFold(CFG.n_fold, random_state=42, shuffle=True) #rs = 42

for k, (_, test_idx) in enumerate(split.split(train, train.sign)):
    train.loc[test_idx, 'fold'] = k

train.fold = train.fold.astype(int)
display(train.groupby('fold').size())
     
    
inter_dallata = train[train.fold==0].reset_index(drop=True)

cnn, tran1, tran2 = [], [], []
for i in tqdm(range(len(inter_dallata))):
    sample = inter_dallata.loc[i] # 45 len=225

    yy = load_relevant_data_subset(base_dir + sample['path'])
    md_st = time.time()
    #output = prediction_fn(inputs=yy)['outputs']
    output = prediction_fn_cnn(inputs=yy)['eff'] #['eff'] ['rex']
    output2 = prediction_fn_trans11(inputs=yy)['outputs']
    output3 = prediction_fn_trans22(inputs=yy)['outputs']
    cnn.append(output)
    tran1.append(output2)
    tran2.append(output3)
    
cnn2 = np.array(cnn)
tran1 = np.array(tran1)
tran2 = np.array(tran2)

from sklearn.metrics import accuracy_score

intr = inter_dallata.copy()
ranges = np.arange(0, 1.01, 0.01)
ranges2 = np.arange(0, 1.05, 0.05)
labs = inter_dallata.label.values

best_score = 0
best_coefs = (None, None)
for ran in ranges:
    for ran2 in ranges2:
        k_cnn = ran
        k_tran = 1 - k_cnn
        
        k_in = ran2
        k_in2 = 1 - k_in

        prds = cnn2 * k_cnn + k_tran * (tran1 * k_in + tran2 * k_in2)
        acc = accuracy_score(labs, prds.argmax(2))

        if acc > best_score:
            print(f'cnn: {k_cnn} | tran {k_tran} | bert {k_in} and deberta {k_in2}')
            print(f'score: {acc:.5f}')
            print(f'Improve {best_score:.5f} --> {acc:.5f}')
            best_score = acc
            best_coefs = k_cnn, k_tran, k_in, k_in2
            print()

print(); print()
print(best_score, best_coefs)'''

'\nimport json\ndef read_dict(file_path):\n    path = os.path.expanduser(file_path)\n    with open(path, "r") as f:\n        dic = json.load(f)\n    return dic\n\nclass CFG:\n    base_path = \'/kaggle/input/\'\n    fold_group = False\n    n_fold = 8 #8\ntrain = pd.read_csv(\'/kaggle/input/asl-signs/train.csv\')\nlabel_index = read_dict(f"{CFG.base_path}/asl-signs/sign_to_prediction_index_map.json")\nindex_label = dict([(label_index[key], key) for key in label_index])\ntrain["label"] = train["sign"].map(lambda sign: label_index[sign])\nprint(train.shape)\ndisplay(train.head())\n\nimport numpy as np\nfrom sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold\n\nsplit = StratifiedKFold(CFG.n_fold, random_state=42, shuffle=True) #rs = 42\n\nfor k, (_, test_idx) in enumerate(split.split(train, train.sign)):\n    train.loc[test_idx, \'fold\'] = k\n\ntrain.fold = train.fold.astype(int)\ndisplay(train.groupby(\'fold\').size())\n     \n    \ninter_dallata = train[train.fold==0].r

In [17]:
# 8986  0.904233700254022 (0.58, 0.42000000000000004, 0.8500000000000001, 0.1499999999999999)
# 8975  0.9022015241320914 (0.56, 0.43999999999999995, 0.8500000000000001, 0.1499999999999999)
# 8966  0.9022015241320914 (0.5700000000000001, 0.42999999999999994, 0.45, 0.55)